In [1]:
import tensorwatch as tw
from dlc_lib import DlcDataset
ds = DlcDataset(linearize=True, data_root='D:\\datasets\\dlc\\current')
ds = tw.pyt_ds2list(ds, 100)

Using data in D:\datasets\dlc\current


In [2]:
import tensorwatch as tw

In [3]:
import numpy as np
import matplotlib.pyplot as plt
def show_image(img, size=None):
    if size is not None or (hasattr(img, 'shape') and len(img.shape) == 1):
        if size is None:
            # make guess for 1-dim tensors
            h = int(math.sqrt(img.shape[0]))
            w = int(img.shape[0] / h)
            size = h,w
        img = np.reshape(img, size)
    #fig = plt.Figure()
    #ax = fig.add_subplot(111)
    #ax.imshow(img)
    return plt.imshow(img) #TODO support console

In [4]:
cap_imgs = tw.col2array(ds, 0)

In [5]:
results = tw.search_similar([ds[3][0]], cap_imgs, topk=3)

In [6]:
img_plot_data = [(data, str(i) + '-' + str(score), ds[i][1]) for score, (i, input, data) in results[0]]

In [8]:
img_plot = tw.open(img_plot_data, type='image', rows=2, cols=2, img_width=39, img_height=69, viz_img_scale=3, height='3in')
img_plot.show()